In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn import model_selection, preprocessing
import xgboost as xgb
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [ ]:
train_df = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
test_df = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
macro_df = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])
train_df = pd.merge(train_df, macro_df, how='left', on='timestamp')
test_df = pd.merge(test_df, macro_df, how='left', on='timestamp')
print(train_df.shape, test_df.shape)

In [ ]:
train_df.head()

In [ ]:
# we will add here some data cleaning

for f in train_df.columns:
    if train_df[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f].values)) 
        train_df[f] = lbl.transform(list(train_df[f].values))
        
train_df = train_df.fillna(0)
print ("Just to override previous one")


In [ ]:
from sklearn.cross_validation import KFold
kf = KFold(len(train_df), n_folds=5, random_state=555, shuffle=True)


# we will use it later...
#skf = StratifiedKFold(n_splits=2)
#skf.get_n_splits(X, y)

features = list(train_df.columns)
features_remove = ['id', 'price_doc', 'timestamp']
features_m = [x for x in features if x not in features_remove]

#print (train_df.dtypes[:])
# Get only numerical variables
features_numbers = [features_m[i] for i in range(len(features_m)) if train_df.dtypes[i] == "int64" or train_df.dtypes[i] == "float64"]
#print (features_numbers)

In [ ]:
for itrain, itest in kf:
    clf = linear_model.Lasso(alpha=0.1, normalize = True)

    X_train, y_train = train_df.loc[itrain, features_numbers], train_df.loc[itrain, 'price_doc']
    X_test, y_test = train_df.loc[itest, features_numbers], train_df.loc[itest, 'price_doc']
    
    print (X_train.values)
    
    #clf.fit(X_train.values, y_train)
    
    # now we can train and test on cross-validated data!